In [ ]:
from flask import Flask, make_response, request, render_template
import io
from io import StringIO
import pandas as pd
import numpy as np 
import pickle
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression

app = Flask(__name__)

def feature_engineering(data):
    
    #convert sex column into 0 and 1 , if male then 1 else 0
    # data['Sex'] = np.where(data['Sex'] == " Male", 1, 0)
    data.columns=['Age', 'Education', 'Education Yrs', 'Marital Status', 'Employer',
       'Occupation', 'Relationship Status', 'Sex', 'Capital Gain',
       'Capital Loss', 'Hours per Week', 'Native Country', 'Salary']
    
    data.drop('Relationship Status',axis=1,inplace=True)
    data.drop('Age',axis=1,inplace=True)
    data.drop('Sex',axis=1,inplace=True)
    data.drop('Capital Loss',axis=1,inplace=True)
    data.drop('Capital Gain',axis=1,inplace=True)
    #occupation
    data['Occupation'] = np.where(data['Occupation'] == ' ?','Missing',data['Occupation'])
    label_enco_occu = {value: key for key, value in enumerate(data['Occupation'].unique())}
    data['Occupation'] = data['Occupation'].map(label_enco_occu)
    
    label_enco_marital_status = {value: key for key, value in enumerate(data['Marital Status'].unique())}
    data['Marital Status'] = data['Marital Status'].map(label_enco_marital_status)
    
    
    label_enco_edu = {value: key for key, value in enumerate(data['Education'].unique())}
    data['Education'] = data['Education'].map(label_enco_edu)
    
    #Employer
    data['Employer'] = np.where(data['Employer'] == ' ?','Missing',data['Employer'])
    label_enco_employee = {value: key for key, value in enumerate(data['Employer'].unique())}
    data['Employer'] = data['Employer'].map(label_enco_employee)
    
    #Native Country
    data['Native Country'] = np.where(data['Native Country'] == ' ?','Missing',data['Native Country'])
    label_enco_country = {value: key for key, value in enumerate(data['Native Country'].unique())}
    data['Native Country'] = data['Native Country'].map(label_enco_country)
    
    #Relationship Status
#     label_enco_edu = {value: key for key, value in enumerate(data['Relationship Status'].unique())}
#     data['Relationship Status'] = data['Relationship Status'].map(label_enco_edu)
    return data

def scalar(data):
    sc = StandardScaler()
    X = data[[ 'Education', 'Education Yrs', 'Marital Status', 'Employer',
       'Occupation', 'Hours per Week', 'Native Country']]
    X = sc.fit_transform(X)
    return (X)

@app.route('/', methods=['GET'])
def Home():
    return render_template('index.html')

@app.route('/predict', methods=['POST'])
def predict():
    f = request.files['data_file']
    if not f:
        return render_template('index.html', prediction_text="No file selected")
    
    stream = io.StringIO(f.stream.read().decode("UTF8"), newline = None)
    result = stream.read()
    data = pd.read_csv(StringIO(result))
    
    #feature engineering
    data = feature_engineering(data)
    
    X = scalar(data)
    
    #load tha model from disk
    loaded_data = pickle.load(open("model1.pkl",'rb'))
    
    print(loaded_model)
    
    result = loaded_model.predict(X)
    
    return render_template('index.html',prediction_text = "Predicted Salary is/are: {}".format(result))

if __name__ == "__main__":
    app.run(debug=False,port=8000)
    
